In [1]:
import pickle
import os
from numpy import random
import wfdb

import sys
sys.path.append('../libs')

import flacdb
import prepare_data

In [2]:
def get_downloaded_rec_segs():
    rec_segs = os.listdir(flacdb.ROOT)
    to_tuple = lambda i: flacdb.rec_seg_to_tuple(i.split('.')[0])
    S1 = {to_tuple(i) for i in rec_segs if '_x.flac' in i}
    S2 = {to_tuple(i) for i in rec_segs if '_x.hea' in i}
    S3 = {to_tuple(i) for i in rec_segs if '_y.hea' in i}
    S4 = {to_tuple(i) for i in rec_segs if '_y.flac' in i}
    S5 = {to_tuple(i) for i in rec_segs if '.hea' in i and '_x' not in i and '_y' not in i}
    S = S1 & S2 & S3 & S4 & S5
    return S

In [53]:
def get_available_rec_segs(): 
    S = []
    db = 'mimic3wdb'
    recs = wfdb.get_record_list(db)
    for rec in recs[:1000]:
        S += wfdb.get_record_list(db + '/' + rec)
    to_tuple = lambda i: flacdb.rec_seg_to_tuple(i.split('.')[0])
    S = {flacdb.rec_seg_to_tuple(i) for i in S if '_' in i}
    return S

In [54]:
%%time
S_available = get_available_rec_segs()

CPU times: user 5.63 s, sys: 188 ms, total: 5.82 s
Wall time: 3min 40s


In [42]:
S_ = wfdb.get_record_list('mimic3wdb')
S_ = {int(i.split('/')[1]) for i in S_}

In [44]:
all(i[0] in S_ for i in rec_segs)

True

In [3]:
hdrs = pickle.load(open('/scr1/mimic/headers_.pkl', 'rb'))
for i in hdrs:
    if i['sig_name'] is None:
        i['sig_name'] = []
bp_sigs = ['ABP', 'CVP', 'PAP', 'ICP']
hdrs_filtered = [
    i for i in hdrs
    if any(j in i['sig_name'] for j in bp_sigs)
    and 'n' not in i['record_name']    
]
S = {flacdb.rec_seg_to_tuple(i['record_name']) for i in hdrs_filtered}

In [5]:
hdrs_m = pickle.load(open('/scr1/mimic/headers_matched_.pkl', 'rb'))
S_m = {
    flacdb.rec_seg_to_tuple(i['record_name']) 
    for i in hdrs_m if i['record_name'][0] != 'p'
}

In [7]:
len(S_m - S)

531338

In [6]:
len(S - S_m)

401217

In [3]:
def get_rec_segs_with_bp():
    hdrs = pickle.load(open('/scr1/mimic/headers_.pkl', 'rb'))
    for i in hdrs:
        if i['sig_name'] is None:
            i['sig_name'] = []
    bp_sigs = ['ABP', 'CVP', 'PAP', 'ICP']
    hdrs_filtered = [
        i for i in hdrs
        if any(j in i['sig_name'] for j in bp_sigs)
        and 'n' not in i['record_name']    
    ]
    S = {flacdb.rec_seg_to_tuple(i['record_name']) for i in hdrs_filtered}
    return S

In [4]:
def get_matched_rec_segs():
    hdrs = pickle.load(open('/scr1/mimic/headers_matched_.pkl', 'rb'))
    S = {
        flacdb.rec_seg_to_tuple(i['record_name']) 
        for i in hdrs if i['record_name'][0] != 'p'
    }
    return S

In [21]:
S_has_bp = get_rec_segs_with_bp()
S_matched = get_matched_rec_segs()
S_downloaded = get_downloaded_rec_segs()
S = (S_has_bp | S_matched) - S_downloaded
rec_segs = sorted(S)
len(rec_segs)

705272

In [27]:
_to_file = lambda x: '{}/{}_{}'.format(x[0], x[0], str(x[1]).zfill(4))
_to_files = lambda x: [_to_file(x) + '.dat', _to_file(x) + '.hea']
filenames = [j for i in rec_segs for j in _to_files(i)]

In [29]:
for part in range(10):
    with open('/scr1/mimic/files_lists/part_{}.txt'.format(part), 'w') as f:
        f.write('\n'.join([i for i in filenames if i[1] == str(part)]))